# Benchmark utilities.

The present notebook explains how to use the module *benchmark_utils* from **QQuantLib.utils** package. In this module several functions help the user to automatize the creation of price estimation problems

Following functions (from **benchmark.benchmark_utils**) will be explained:

1. *combination_for_dictionary*
2. *combination_for_list*
3. *create_pe_problem*
4. *create_ae_pe_solution*

In [ ]:
import sys
sys.path.append("../../")
import numpy as np
import pandas as pd
import itertools as it
import matplotlib.pyplot as plt
import json

## 1. combination_for_dictionary function


The input of this function is a Python dictionary where for each key the values **MUST BE**  a list. For a given key the correspondent list can have one or more elements. For each element of the list, a different output dictionary will be generated. If different keys have corresponding lists with more than one element all possible combinations will be returned by the function.

In the following cells, we give an example of how this works:

In [ ]:
from QQuantLib.utils.benchmark_utils import combination_for_dictionary

We are going to start with a *probability_dictionary* configuration where we have the following list with more than one element:

* *risk_free_rate*  with 2 possible values
* *volatility*: with 3 possible values.

The **combination_for_dictionary** will create a list of Python dictionaries where all the possibilities will be exhausted: in this case, we will have *risk_free_rate*volatility* number of dictionaries: 2*3=6

In [ ]:
probability_dictionary = {
    'probability_type': ['Black-Scholes'],
    's_0': [1],
    'risk_free_rate': [0.02, 0.05],
    'maturity': [1.0],
    'volatility': [0.1, 0.3, 0.5]
}
list_of_probabilities = combination_for_dictionary(probability_dictionary)
print(len(list_of_probabilities))

In [ ]:
list_of_probabilities[0]

## 2 combination_for_list function

The input of this function is a list of Python dictionaries. Each dictionary should have a key-value pair where the value is a list. This function iterates over each dictionary of the input list, generates all the possible combinations of the dictionary (using the *combination_for_dictionary* function) and finally concatenates all of them.

The following cells show how this function works.

In [ ]:
from QQuantLib.utils.benchmark_utils  import combination_for_list

In the following case, we will have 2 different payoffs (a *European_Put_Option* and a *Futures*) and we want to test several *strikes*. So the *combination_for_list* will create all the possible configurations for each dictionary of the list and concatenate them.

So we will have 2 possible dictionaries from the *European_Put_Option* one and 3 combinations from the *futures* one. So our final list will have 5 possible dictionaries.

In [ ]:
payoffs = [
    {
        'pay_off_type': ['European_Put_Option'],
        'strike': [1.5, 2.0],
        'coupon': [None]
    },
    {
        'pay_off_type': ['Futures'],
        'strike': [0.5, 1.0, 1.5], 
        'coupon': [None]}
]
list_for_payoffs = combination_for_list(payoffs)
len(list_for_payoffs)

In [ ]:
list_for_payoffs[4]

## 3. create_pe_problem function

We are going to define a complete **price estimation problem** as a dictionary with all the complete information for generating an option price estimation problem as explained in **/misc/notebooks/12_ApplicationTo_Finance_03_AEPriceEstimation**. This kind of dictionary will need the following keys:

* domain_configuration keys: related to the domain of the price estimation problem:
    * x0
    * xf
    * n_qbits
* probability_configuration keys: related to the kind of density probability, asset information and market information:
    * probability_type
    * s_0
    * risk_free_rate
    * maturity
    * volatility
* payoff_configuration keys: related to the configuration of derivative option:
    * pay_off_type
    * strike
    * coupon
    
The **create_pe_problem** function was implemented for automatizing the creation of this **price estimation problem**. 

The input of this function will be the following three list of dictionaries:

* domain_cfg: list of Python dictionaries with different domain configurations.
* payoff_cfg: list of Python dictionaries with different payoff configurations. 
* density_cfg: list of Python dictionaries with different probability configurations.

The values of the different dictionaries **MUST BE** a list with one or more elements. The following cell shows an example of the typical input list of dictionaries for the **create_pe_problem** function:

In [ ]:
domain_cfg = [{'x0': [0.01], 'xf': [5.0], 'n_qbits': [5]}]
density_cfg = [{
    'probability_type': ['Black-Scholes'], 's_0': [1], 'risk_free_rate': [0.05],
    'maturity': [1.0], 'volatility': [0.5]
}]
payoff_cfg = [{'pay_off_type': ['European_Call_Option'], 'strike': [0.5], 'coupon': [None]}]

The **create_pe_problem** function allows to create all the possible **price estimation** problems combinations from the inputs.

In [ ]:
from QQuantLib.utils.benchmark_utils import create_pe_problem

In [ ]:
#In this example only one complete price estimation problem is created.
pe_problem = create_pe_problem(domain_cfg, density_cfg, payoff_cfg)

In [ ]:
pe_problem

If any of the keys of any of the inputs have more than one element then all possible combinations will be generated:

In [ ]:
#Here we have a list with 2 elements
domain_cfg = [{'x0': [0.01], 'xf': [5.0], 'n_qbits': [5, 7]}]
#And here another two elements
density_cfg = [{
    'probability_type': ['Black-Scholes'], 's_0': [1], 'risk_free_rate': [0.05, 0.1],
    'maturity': [1.0], 'volatility': [0.5]
}]

#In this case the function generates 4 possible price estimation problems
pe_problem = create_pe_problem(domain_cfg, density_cfg, payoff_cfg)
print(len(pe_problem))

In [ ]:
pe_problem[0]

In [ ]:
pe_problem[3]

The inputs of the *create_pe_problem* are list of dictionaries so for each input we can provide several dictionaries. The function will return all the possible combinations again!!

In [ ]:
#In this case we have 7 different payoffs
payoff_cfg = [
    {'pay_off_type': ['European_Call_Option'], 'strike': [0.5], 'coupon': [None]},
    {'pay_off_type': ['European_Put_Option'], 'strike': [1.5], 'coupon': [None]},
    {'pay_off_type': ['Futures'], 'strike': [0.5, 1.0, 1.5], 'coupon': [None]},
    {'pay_off_type': ['Digital_Call_Option'], 'strike': [0.5], 'coupon': [1.0]},
    {'pay_off_type': ['Digital_Put_Option'], 'strike': [1.5], 'coupon': [1.0]}]

#The domain cfg have 2 different domain configurations
#The density_cfg have 2 different probability configurations
#So 2*2*7=28 different price estimation problems will be generated
pe_problem = create_pe_problem(domain_cfg, density_cfg, payoff_cfg)
print(len(pe_problem))

## 4. create_ae_pe_solution_list function
 
Now, we have a complete list of price estimation problems (**pe_problem**) and we want to solve each one with different **AE** algorithms and configurations. 

Let's imagine, for example, we have the following **AE** configuration scheme:



In [ ]:
ae_config = [
    {
        "ae_type": ["IQAE"],

        "epsilon": [1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5, 1.0e-6],

        "alpha": [0.05],

        "gamma": [None],
        "q": [None],

        "encoding" : [0, 2],
        "multiplexor":  [True],

        "mcz_qlm": [False],
        "file": ["IQAE"],
        "shots": [100]

    },
    {
        "ae_type": ["RQAE"],

        "epsilon": [1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5, 1.0e-6],

        "alpha": [None],

        "gamma": [0.05],
        "q": [2, 5, 10],
        
        "encoding" : [2],
        "multiplexor":  [True],

        "mcz_qlm": [False],
        "file": ["RQAE"],
        "shots": [100]
    },
]

So we want to execute each one of the 27 price estimations problems with each possible **AE** configuration from **ae_config**.

First, we can use *combination_for_list* function to create all the possible **AE** configurations that we want to use. In this case will be:

* 10 configurations for the first dictionary (IQAE one)
* 15  configurations for the second dictionary (RQAE one)

So in total 10 + 15 = 25 combibnations

In [ ]:
ae_combinations = combination_for_list(ae_config)
print(len(ae_combinations))

Now we can combine the 27 different price estimation problems with all the 25 different **AE** configurations using the *create_ae_pe_solution* function. So in this case we are going to generate: 25 * 27 = 700 complete Amplitude Estimation price problems

In [ ]:
from QQuantLib.utils.benchmark_utils import create_ae_pe_solution

In [ ]:
final_list = create_ae_pe_solution(ae_combinations, pe_problem)

In [ ]:
len(final_list)